In [2]:
import datatable as dt
import matplotlib.pyplot as plt

X_train = dt.fread('X_train.csv').to_numpy()
y_train = dt.fread('y_train.csv').to_numpy()
df_test = dt.fread('X_test.csv').to_numpy()

TypeError: Frame cannot be converted into a numpy array because it has columns of incompatible types

In [3]:
import datatable as dt
import matplotlib.pyplot as plt

X_train = dt.fread('X_train.csv')
y_train = dt.fread('y_train.csv')
df_test = dt.fread('X_test.csv')

In [5]:
X_train.head()

,customer_id,imputed__no_of_children,imputed__no_of_days_employed,imputed__total_family_members,imputed__migrant_worker,imputed__yearly_debt_payments,imputed__credit_score,one_hot__x0_M,one_hot__x1_Y,one_hot__x2_Y,…,scaled__total_family_members,scaled__yearly_debt_payments,scaled__credit_limit,scaled__credit_limit_used(%),scaled__credit_score
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CST_115179,0,612,1,1,33070.3,544,0,0,1,…,−1.26947,0.0738169,−0.166082,0.706761,−2.37369
1,CST_121920,0,2771,2,0,15329.5,857,1,0,1,…,−0.17363,−0.954722,−0.0387526,−0.00794164,0.73754
2,CST_109330,0,204,2,0,48416.6,650,1,0,1,…,−0.17363,0.963536,−0.0130039,−0.314243,−1.32005
3,CST_128288,0,11941,2,0,22574.4,754,0,0,1,…,−0.17363,−0.534695,−0.0729498,−1.09701,−0.286283
4,CST_151355,0,1459,1,0,38282.9,927,1,1,1,…,−1.26947,0.376027,0.0628576,0.774828,1.43334
5,CST_123268,0,2898,2,1,37046.9,937,0,1,0,…,−0.17363,0.304363,−0.0220434,−1.13105,1.53274
6,CST_127502,1,5541,3,0,50839.4,733,1,0,1,…,0.922211,1.104,−0.0149236,−0.348276,−0.495024
7,CST_151722,1,1448,3,0,30008.5,906,0,1,1,…,0.922211,−0.103695,−0.0757467,1.31936,1.2246
8,CST_111670,2,2791,4,0,9509.1,666,0,1,1,…,2.01805,−1.29217,−0.101031,−1.30121,−1.16101


In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
def to_tensor_X_y(X, y=None):
    '''Returns a data set of numpy array to X and y torch tensors '''
    X = torch.from_numpy(X).type(torch.float)
    if y:
        y = torch.from_numpy(y).type(torch.float)
        y = torch.reshape(y, (-1, 1))
    return X, y


X_train, y_train = to_tensor_X_y(X_train, y_train)
X_test = to_tensor_X_y(df_test)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Split the training data to training and validation sets
data_train = TensorDataset(X_train, y_train)
data_test = TensorDataset(X_test)
train_ratio = 0.8
train_size = int(len(data_train) * train_ratio)
val_size = len(data_train) - train_size
train_ds, val_ds = random_split(data_train, [train_size, val_size])

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size = 32

# Training sampler and data loader
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)
test_loader = DataLoader(data_test, batch_size)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

input_size = X_train.shape[1]
output_size = y_train.shape[1]


class My_Model(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, input_size//4)
        self.linear2 = nn.Linear(input_size//4, input_size//2)
        self.output = nn.Linear(input_size//2, output_size)

    def forward(self, X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        return self.output(X)

    def training_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        y_pred = self(inputs)
        # Calculate loss
        loss = F.binary_cross_entropy(y_pred, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        y_pred = self(inputs)
        # Calculate loss
        loss = F.binary_cross_entropy(y_pred, targets)
        return {'val_loss': loss.detach()}  # Detach = no_grad

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 10% of num_epochs
        if (epoch + 1) % (num_epochs // 10) == 0:
            print(f"Epoch [{epoch + 1}], val_loss: {result['val_loss']:,.4f}")

In [ ]:
import math
from tqdm import tqdm


def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    last_loss = -math.inf
    patience = epochs // 20
    for epoch in tqdm(range(epochs)):
        trigger_times = 0
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)

        # Early stopping
        if result['val_loss'] > last_loss:
            trigger_times += 1
            if trigger_times >= patience:
                history.append(result)
                return history
        last_loss = result['val_loss']

        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [ ]:
model = My_Model(input_size, output_size)
epochs = int(1e4)
lr = 1e-5
history = fit(epochs, lr, model, train_loader, val_loader)

## This model is chosen because of its smaller validation loss against those with Optuna Search

In [ ]:
# Plotting
losses = [x['val_loss'] for x in history]
plt.plot(losses, '-x')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Loss vs. No. of epochs')

In [ ]:
import pandas as pd

# Make predictions using the trained model
def to_csv(dataset, model, file_name):
    preds, actual = [], []
    for X, y in dataset:
        X = X.unsqueeze(0)
        predictions = model(X)
        prediction = predictions[0].detach()
        preds.append([i for i in prediction.flatten().numpy()])
        actual.append([i for i in y.numpy().flatten()])
    preds = [item for sublist in preds for item in sublist]
    predicted_df = pd.DataFrame(data={'predicted_value': preds})
    predicted_df.to_csv(file_name)

In [ ]:
to_csv(test_loader, model, 'Predicted value from Pytorch Linear model.csv')